In [1]:
import tensorflow_hub as hub
from utils.augmentation import preprocess_image
from loader.mimic_cxr_jpg_loader import MIMIC_CXR_JPG_Loader
from utils.analysis import *
from IPython.display import Image
from loader.labels import LABELS

BATCH_SIZE = 10

# VARIABLES
project_folder = os.getcwd()
if project_folder.endswith('/job'):
  project_folder = project_folder[:-4]
BASE_MODEL_PATH = './base-models/remedis/cxr-152x2-remedis-m/'
hub_path = os.path.join(project_folder, BASE_MODEL_PATH)
IMAGE_SIZE = (448, 448)
START_TIME = get_curr_datetime()

# Load hub
module = hub.load(hub_path)

# Pathology: The image is of shape (<BATCH_SIZE>, 224, 224, 3)
# Chest X-Ray: The image is of shape (<BATCH_SIZE>, 448, 448, 3)


def _preprocess_val(x, y, info=None):
    x = preprocess_image(
        x, *IMAGE_SIZE,
        is_training=False, color_distort=False, crop='Center')
    return x, y

# load image
customLoader = MIMIC_CXR_JPG_Loader({'train': 0, 'validate': 0, 'test': 2*BATCH_SIZE}, project_folder)
train_tfds, val_tfds, test_tfds = customLoader.load()
val_tfds = val_tfds.shuffle(buffer_size=BATCH_SIZE)
batched_test_tfds = test_tfds.map(_preprocess_val).batch(BATCH_SIZE)

for batch in batched_test_tfds:
    image = batch[0]
    gt = batch[1]

    show_prediction(*batch, module, os.path.join(project_folder, './out/figs'), START_TIME)
    for i in range(BATCH_SIZE):
        # Display the image
        
        display(image[i])
        print('gt:', LABELS[np.where(gt[i].numpy() == 1)])
        print('\n')
    break


embedding_of_image = module(image)
print('embeddings:')

2024-02-14 19:46:45.846219: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-14 19:46:46.530914: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.2/lib64:/home/payam/miniconda3/envs/tf2-gpu/lib/
2024-02-14 19:46:46.531002: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.2/lib64:/home/payam/miniconda3/envs/tf2-gpu

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run Identity: Dst tensor is not initialized. [Op:Identity]

In [2]:
!echo $LD_LIBRARY_PATH

/usr/local/cuda-12.2/lib64:/home/payam/miniconda3/envs/tf2-gpu/lib/
